In [1]:
import pandas as pd

from src.SearchEngine import SearchEngine
engine = SearchEngine()

c:\Users\t_zim\miniconda3\envs\bible_app\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Execution

In [7]:
response = engine.tab1_engine(question = "who is my father?",
                              run_cross_encoder = "Yes",
                              testament = "All",
                              k=100,
                              relevance=.50)

Cross Encoding...


In [8]:
response

,book,chapter,text,similarity score,cross_encoder_score
712,John,15,"I am the true vine, and my Father is the husba...",0.309137,0.706877
711,John,14,Let not your heart be troubled: ye believe in ...,0.400252,-0.068318
328,Ephesians,1,"Paul, an apostle of Jesus Christ by the will o...",0.267195,-0.971882
96,1 Samuel,20,"And David fled from Naioth in Ramah, and came ...",0.354515,-0.987150
266,Colossians,1,"Paul, an apostle of Jesus Christ by the will o...",0.281199,-1.741883
...,...,...,...,...,...
1063,Psalm,95,"O come, let us sing unto the LORD: let us make...",0.268457,-9.519678
984,Psalm,16,"{Michtam of David.} Preserve me, O God: for in...",0.276880,-9.709694
1080,Psalm,112,Praise ye the LORD. Blessed <i>is</i> the man ...,0.253865,-9.829615
998,Psalm,30,{A Psalm <i>and</i> Song <i>at</i> the dedicat...,0.249807,-10.200428


In [4]:
response

,book,chapter,text,similarity score
711,John,14,Let not your heart be troubled: ye believe in ...,0.400252
1163,Song of Solomon,3,By night on my bed I sought him whom my soul l...,0.398384
238,Acts,10,There was a certain man in Caesarea called Cor...,0.361350
96,1 Samuel,20,"And David fled from Naioth in Ramah, and came ...",0.354515
503,Hebrews,1,"God, who at sundry times and in divers manners...",0.349659
